# Scraping web data and text mining for analysis, and for use with a Tensorflow RNN example

In [14]:
import os
os.getcwd()

'/Users/MF/Desktop/data-science-side-projects/data_scraping_project'

In [5]:
#from HTMLParser import HTMLParser
#parser = HTMLParser()
#parser.feed('https://docs.python.org/2/library/htmlparser.html')

In [10]:
#%lsmagic
#!ps -a
##%time
##2+2

Example using scrapy module for data scraping, see: https://doc.scrapy.org/en/1.4/intro/overview.html

In [17]:
with open('quotes_spider.py', 'w') as swf:
    swf.write('''\
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/tag/humor/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.xpath('span/small/text()').extract_first(),
            }

        next_page = response.css('li.next a::attr("href")').extract_first()
        if next_page is not None:
            yield response.follow(next_page, self.parse)
            ''')
    swf.close()

In [20]:
! scrapy runspider quotes_spider.py -o quotes.json &>/dev/null